In [255]:
import numpy as np
import pandas as pd
import os
import requests
import bs4
import json
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from difflib import SequenceMatcher
import project04 as proj

In [256]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [257]:
url = 'https://genius.com{}'
artist = 'taylor swift'
search_bar = '/artists/'+artist.capitalize().replace(' ', '-')

In [258]:
response = requests.get(url.format(search_bar))
soup = bs4.BeautifulSoup(response.content)

In [259]:
for i in soup.find_all('a', attrs={'class': 'vertical_album_card'}):
    print(i['title'])

evermore (digitally autographed fan edition)
evermore (deluxe version)
evermore (Japanese Edition)
evermore
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore: the saltbox house chapter


In [260]:
new_url = soup.find_all('a', attrs={'class': 'full_width_button'})[1]['href']

In [261]:
albums_request = requests.get(url.format(new_url))
albums = bs4.BeautifulSoup(albums_request.content)

In [262]:
album_lst = albums.find('ul', attrs={'class': 'album_list primary_list'}).find_all('li')
album_lst

[<li><a class="album_link" href="/albums/Taylor-swift/Taylor-s-songs-2003-demo">"Taylor's Songs" 2003 Demo</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/2003-demo-cd-two-songs">2003 Demo CD (Two Songs)</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/2004-demo-cd-three-songs">2004 Demo CD (Three Songs)</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/2004-demo-cd">2004 Demo CD</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/Evermore">evermore</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/Red">Red</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/Fearless-international-edition">Fearless (International Edition)</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/Spotify-singles">Spotify Singles</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/Red-deluxe-edition">Red (Deluxe Edition)</a></li>,
 <li><a class="album_link" href="/albums/Taylor-swift/1989-taylors-version"

In [278]:
song_dict = {}
for i in album_lst:
    if i.text == 'Unreleased Songs':
        continue
    link = i.find('a')['href']
    new_url = url.format(link)
    response = requests.get(new_url)
    soup = bs4.BeautifulSoup(response.content)
    if soup.find('strong'):
        continue
    tracklist = soup.find_all('div', attrs={'class': 'column_layout-column_span column_layout-column_span--primary'})[1]
    for j in tracklist.find_all('div', attrs={'class': 'chart_row-content'}):
        link = j.find('a')['href']
        song_link = requests.get(link)
        song = bs4.BeautifulSoup(song_link.content)
        if song.title.text == 'Application Error':
            continue
        song_page = song.find('div', attrs={'id': 'application'})
        title = song.find('h1').text.replace('\u200b', '')
        singer = song.find_all('a')[8].text.lower()
        if singer != artist:
            continue
        if song_page == None:
            continue
        else:
            lyr = song_page.find_all('div')[39].text
            if lyr[:20] == 'How to Format Lyrics':
                lyr = song_page.find_all('div')[40].text
            lyr = re.sub(r'([a-z])([A-Z])', r'\1 xxxx \2', lyr)
            lyr = re.sub(r'\[[\w :&-]*\]', ' ', lyr)
            lyr = re.sub(r'Embed[\w .]*', '', lyr)
            lyr = lyr.replace('\u205f', ' ')
            lyr = lyr.replace('\u2005', ' ')
            lyr = re.sub(r'(\)|\?)(\'|\w)', r'\1 \2', lyr)
            lyr = re.sub(r'([a-zA-z])([0-9]+)', r'\1', lyr)
            lyr = lyr.replace('TAOO', 'TA13OO')
            lyr = lyr.strip()
        if title in song_dict:
            continue
        else:
            song_dict[title] = lyr

In [281]:
lyrics = list(song_dict.values())

In [282]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(), columns=['idf_weights'])

In [283]:
count_vector = cv.transform(lyrics)
tf_idf_vector = tfidf_transformer.transform(count_vector)

In [284]:
feature_names = cv.get_feature_names()
first_document_vector = tf_idf_vector[90]
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=['tfidf'])
df.sort_values(by=['tfidf'], ascending=False).head(20)

,tfidf
my,0.481628
you,0.337579
forever,0.229584
xxxx,0.227637
take,0.211677
re,0.194225
go,0.187805
we,0.187299
ever,0.184480
and,0.167204


In [285]:
songs = ' xxxx '.join(lyrics)

In [286]:
tokens = proj.tokenize(songs)

In [287]:
unigram = proj.NGramLM(3, tuple(tokens))

In [302]:
samp = unigram.sample(500)

In [303]:
samp = samp.replace(' \' ', '\'')
samp = samp.replace('xxxx', '\n')
samp = re.sub(' , ', ', ', samp)
samp = samp.replace(' . ', '. ')
samp = samp.replace(' ( ', ' (')
samp = samp.replace(' )', ') ')
samp = samp.replace(' ? ', '? ')

In [304]:
samp = samp.strip('\x02 ')

In [305]:
lines = samp.split('\n')
clean_samp = []
for i in lines:
    i = i.strip()
    i = i[0].upper() + i[1:]
    clean_samp.append(i)
samp = '\n'.join(clean_samp)

In [306]:
print(samp)

Smokey black nights I'm the onе who understands you
Even though we boarded them up to you
With whatever talent money buys
So here's new, the less I know you better, know you better, that's the reason for the longest time
Are we in the lobby
Candle wax and Polaroids on the floor
Wearing your clothes
All I can feel it now? You should've been
There were sirens in the palm of your body
Laughin'at my feet
You almost ran the red'cause of me as we pass by
Tell them how the leaves
When people say, the second day
When I met you
Just wanna be everything you looked at me
So far above feeling anything at all
And then you called, showed your hand through plastic now " Ooh, ooh (Ah - uh, myself
A song that [? ] We can't you think Tim Mc
Graw so I let them see me clearly
And bring on all my flaws, paranoia, and you talk real slow'til her claws come out
I promise that you're the only one who understands you
Swear to be
Fifteen You sit in class next to her on the street
Flashback to the time of my head